In [ ]:
## Import Libraries
import pandas as pd
import sqlite3
import datetime

In [ ]:
## Read the files

df_Customers = pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_customers_dataset.csv')
df_Sellers = pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_sellers_dataset.csv')
df_OrderReviews= pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_order_reviews_dataset.csv')
df_OrderItems= pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_order_items_dataset.csv')
df_Products= pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_products_dataset.csv')
df_Geolocation= pd.read_csv('/F:/TASK/Project/BI2203/OLIST/olist_geolocation_dataset.csv')
df_ProductCategory= pd.read_csv('F:/TASK/Project/BI2203/OLIST/product_category_name_translation.csv')
df_Orders = pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_orders_dataset.csv')
df_OrderPayments= pd.read_csv('F:/TASK/Project/BI2203/OLIST/olist_order_payments_dataset.csv')

In [ ]:
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_Customers.to_sql("Customers", con =engine)
df_Sellers.to_sql("Sellers", con =engine)
df_OrderReviews.to_sql("OrderReviews", con =engine)
df_OrderItems.to_sql("OrderItems", con =engine)
df_Products.to_sql("Products", con =engine)
df_Geolocation.to_sql("Geolocation", con =engine)
df_ProductCategory.to_sql("ProductCategory", con =engine)
df_Orders.to_sql("OrderDataset", con =engine)
df_OrderPayments.to_sql("OrderPayments", con =engine)

## Run query function
def run_query(a):
    df_sql = pd.read_sql_query(a,con=engine)
    return df_sql

<a id="section-three"></a>
# 3. Business Oriented Data Analysis
<a id="sub-one"></a>
## 3.1. Sale Performance
We will make query that return some essential about Number of sale

In [ ]:
SalePerYear = """
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
	COUNT(order_id) AS NumOfOrders
FROM
	OrderDataset
GROUP BY
	1
ORDER BY
	1
"""
run_query(SalePerYear)

It is apparently seen that number of orders is low in 30/09/2016, 31/10/2016, 31/12/2016, 30/09/2018. They are thus removed and did not affect the further analysis

In [ ]:
SalePerYear = """
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
	COUNT(order_id) AS NumOfOrders
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	1
ORDER BY
	1
"""
run_query(SalePerYear)

 calculate the Average Orders Per Month

In [ ]:
AvgOrdMonth = """
With SaleInf AS(
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH,
	COUNT(order_id) AS NumOfOrders
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	1
ORDER BY
	1
    )
SELECT
	MONTH,
	AVG(NumOfOrders) AS AvgOrdMonth
FROM
	SaleInf
GROUP BY
	MONTH
ORDER BY 2 DESC
"""
run_query(AvgOrdMonth)

As we can see that November has the most orders averages.

More detail about Average revenue of sale per month

In [ ]:
AvgRevMonth = """
With SaleInf AS(
SELECT
    order_id,
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH,
	COUNT(order_id) AS NumOfOrders,
    order_purchase_timestamp AS DATE
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	2,3,5
ORDER BY
	2,3,5
    ),
a AS(
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	1
ORDER BY
	1
),
RealMonth AS(
SELECT MONTH, COUNT(MONTH) AS COUNT
FROM a
GROUP BY MONTH
)

select si.MONTH, SUM(oi.price)/rm.COUNT AS AvgRevMonth
from OrderItems oi
JOIN SaleInf si
ON oi.order_id = si.order_id
JOIN RealMonth rm ON
si.MONTH = rm.MONTH
GROUP BY si.MONTH
ORDER BY 2 desc
"""
run_query(AvgRevMonth)

November also has the highest average revenue.

which category has the highest average monthly sales

In [ ]:
RankCatMonth = """
With SaleInf AS(
SELECT
    order_id,
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH,
	COUNT(order_id) AS NumOfOrders,
    order_purchase_timestamp AS DATE
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	2,3,5
ORDER BY
	2,3,5
    ),
a AS(
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	1
ORDER BY
	1
),
RealMonth AS(
SELECT MONTH, COUNT(MONTH) AS COUNTMONTH
FROM a
GROUP BY MONTH
)
SELECT * 
FROM (SELECT  si.MONTH, 
              pc.product_category_name_english AS Category, 
              SUM(oi.price)/rm.COUNTMONTH AS AvgRevMonth,
              ROW_NUMBER() OVER(PARTITION BY si.MONTH ORDER BY (SUM(oi.price)/rm.COUNTMONTH) DESC) AS RANKCAT
      FROM OrderItems oi
      JOIN SaleInf si ON 
            oi.order_id = si.order_id
      JOIN RealMonth rm ON
            si.MONTH = rm.MONTH
      JOIN Products p ON
            p.product_id = oi.product_id
      JOIN ProductCategory pc ON
            pc.product_category_name = p.product_category_name
      GROUP BY 1,2)
WHERE RANKCAT = 1
"""
run_query(RankCatMonth)

Categories that are repeated a lot like watches_gifts, health_beauty, watches_gifts.

Which city has the highest average monthly sales

In [ ]:
RankCityMonth = """
With SaleInf AS(
SELECT
    order_id,
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH,
	COUNT(order_id) AS NumOfOrders,
    order_purchase_timestamp AS DATE
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	2,3,5
ORDER BY
	2,3,5
    ),
a AS(
SELECT
	strftime("%Y %m",order_purchase_timestamp) AS YEAR,
    strftime("%m",order_purchase_timestamp) AS MONTH
FROM
	OrderDataset
WHERE strftime("%Y %m",order_purchase_timestamp) NOT IN ("2016 09","2016 10","2016 12","2018 09","2018 10")
GROUP BY
	1
ORDER BY
	1
),
RealMonth AS(
SELECT MONTH, COUNT(MONTH) AS COUNT
FROM a
GROUP BY MONTH
)

SELECT * 
FROM (SELECT  si.MONTH, 
              g.geolocation_city AS City, 
              SUM(oi.price)/rm.COUNT AS AvgRevMonth,
              ROW_NUMBER() OVER(PARTITION BY si.MONTH ORDER BY (SUM(oi.price)/rm.COUNT) DESC) AS RANKCITY
      FROM OrderItems oi
      JOIN SaleInf si ON 
            oi.order_id = si.order_id
      JOIN RealMonth rm ON
            si.MONTH = rm.MONTH
      JOIN OrderDataset od ON
            od.order_id = oi.order_id
      JOIN Customers c ON
            c.customer_id = od.customer_id
      JOIN Geolocation g ON
            c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
      GROUP BY 1,2)
WHERE RANKCITY = 1
"""
run_query(RankCityMonth)

Customers from Rio De Janeiro delivers the highest average monthly revenue in all 12 months
<a id="sub-two"></a>
## 3.2. Seller Performance
In this section, we'll take a look at:
- Categories with the most sellers
- Categories with the most average revenue
- Where are the most sellers?

**How many Sellers in each Category? 
What is the average revenue of the seller in each category?**

In [ ]:
CatNumOfSeller = """
SELECT
	pc.product_category_name_english AS Category,
	COUNT(DISTINCT(oi.seller_id)) AS NumOfSeller,
    Sum(oi.price) / COUNT(DISTINCT(oi.seller_id)) AS AvgRevSeller
FROM
	Products p
JOIN ProductCategory pc ON
	p.product_category_name = pc.product_category_name
JOIN OrderItems oi ON
	oi.product_id = p.product_id
GROUP BY 1
ORDER BY 2 DESC
"""
run_query(CatNumOfSeller)

The table above is being sorted in descending order of number of sellers.

There are 71 categories.

The category with the most sellers is health_beauty with 492 Sellers.

The category with the fewest sellers is cds_dvds_musicals with only 1 seller.


In [ ]:
CatNumOfSeller2 = """
SELECT
	pc.product_category_name_english AS Category,
	COUNT(DISTINCT(oi.seller_id)) AS NumOfSeller,
    Sum(oi.price) / COUNT(DISTINCT(oi.seller_id)) AS AvgRevSeller
FROM
	Products p
JOIN ProductCategory pc ON
	p.product_category_name = pc.product_category_name
JOIN OrderItems oi ON
	oi.product_id = p.product_id
GROUP BY 1
ORDER BY 3 DESC
"""
run_query(CatNumOfSeller2)

We can see different insights
Category computers is bringing in the most revenue on average 24773.68 with only 9 sellers

Now still with the table above but we will sort in descending order of Average revenue of seller

**Where are the most sellers?**

Top 10 City with the most Seller

In [ ]:
TopCitySeller = """
SELECT
	g.geolocation_city AS City,
	COUNT(DISTINCT(s.Seller_id)) AS NumOfSeller,
	(100 * COUNT(DISTINCT(s.Seller_id)))/
	(
	SELECT
		COUNT(Seller_id)
	FROM
		Sellers) AS PercOfTotalSeller
FROM
	Sellers s
JOIN geolocation g ON
	s.Seller_zip_code_prefix = g.geolocation_zip_code_prefix
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
"""
run_query(TopCitySeller)

Sellers gather a lot in Sao Paulo (695 people) and São Paulo (678 people)

The total sellers of the two cities above account for 43% of the total number of sellers

<a id="sub-three"></a>
## 3.3. Deliver And Logistics Performance
### 3.3.1. How e-commerce operation process?
